In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Function to display results

In [2]:
def draw_registration_result(source, target, transformation=None, diff_color=True):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if diff_color:
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if transformation is not None:
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

Import point cloud

In [3]:
# source = o3d.io.read_point_cloud("./Data/output/bottle/pcd/bottle_left.ply")
# target = o3d.io.read_point_cloud("./Data/output/bottle/pcd/bottle_right.ply")
source = o3d.io.read_point_cloud("./Data/output/house/pcd/IMG_2630.ply")
target = o3d.io.read_point_cloud("./Data/output/house/pcd/IMG_2631.ply")

Apply random transform

In [12]:
# random transformation matrix
trans_random = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                            [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
source.transform(trans_random)

PointCloud with 291548 points.

In [5]:
draw_registration_result(source, target)

Set threshold for the number of points in the point cloud

In [13]:
threshold = 0.02

In [14]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_random)
print(evaluation)

Initial alignment
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.811153e-05, and correspondence_set size of 291548
Access transformation to get result.


# Global Registration using Fast Point Feature Histograms (FPFH)

To reduce the computational impact we downsample the point clouds and use a voxel grid to reduce the number of points.

In [8]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal) 
    #Noramls are used to calculate the FPFH features
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [9]:
voxel_size=0.00001
# draw_registration_result(source, target, np.identity(4))
print("source points:", len(source.points))
print("target points:", len(target.points))
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)

target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
print("source_down points:", len(source_down.points))
print("target_down points:", len(target_down.points))

source points: 291548
target points: 291301
:: Downsample with a voxel size 0.000.
:: Estimate normal with search radius 0.000.
:: Compute FPFH feature with search radius 0.000.
:: Downsample with a voxel size 0.000.
:: Estimate normal with search radius 0.000.
:: Compute FPFH feature with search radius 0.000.
source_down points: 11293
target_down points: 10092


In [16]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, 
        source_fpfh, target_fpfh, 
        True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n = 3, 
        checkers = [ #Pruning, points that pass the pruning will be subject to RANSAC
            #Checking if the edeges of source and target are about 0.9 of each other
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), 
            #Checking if the distance between the points is less than the threshold
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold) 
        ], 
        criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)) #max_iter, confidence
    return result

In [17]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.000,
   we use a liberal distance threshold 0.000.
RegistrationResult with fitness=3.283450e-01, inlier_rmse=9.188320e-06, and correspondence_set size of 3708
Access transformation to get result.


Draw results with color

In [22]:
draw_registration_result(source_down, target_down, result_ransac.transformation, diff_color=False)

Apply transformation to the source point cloud

In [22]:
source_down.transform(result_ransac.transformation)

PointCloud with 11293 points.

# Local Registration using Iterative Closest Point (ICP)

In [15]:
trans_init = np.eye(4)
reg_p2p = o3d.pipelines.registration.registration_icp(
    source_down, target_down, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(reg_p2p)

RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.184453e-04, and correspondence_set size of 11293
Access transformation to get result.


In [16]:
draw_registration_result(source_down, target_down, reg_p2p.transformation)